In [ ]:
import queue
import threading
import time
import random

def generate_flight_path():
    return random.randint(1, 5)

def generate_priority():
    return random.randint(1, 2)

def generate_pitch_price():
    return round(random.uniform(8, 50), 2)

def generate_start_time():
    start_hour = random.randint(0, 23)
    period = "on-peak" if 7 <= start_hour <= 21 else "off-peak"
    return f"{start_hour:02d}:00", period

def generate_trip_request(user_id):
    flight_path = generate_flight_path()
    pitch_price = generate_pitch_price()
    priority = generate_priority()
    start_time, period = generate_start_time()
    return {
        "User ID": user_id,
        "Flight Path": flight_path,
        "Priority" : priority,
        "Pitch Price": f"£{pitch_price}",
        "Start Time": start_time,
        "Period": period
    }

def trip_request_generator(request_queue, user_count=3, delay_seconds=5):
    user_ids = list(range(1, user_count + 1))
    while True:
        random.shuffle(user_ids)
        for user_id in user_ids:
            request = generate_trip_request(user_id)
            request_queue.put(request)
        time.sleep(delay_seconds)

#def process_requests(request_queue, process_delay=5):
    #try:
       # while True:
           # if not request_queue.empty():
               # request = request_queue.get()
               # print(f"Processing request from User {request['User ID']}: {request}")
               # time.sleep(process_delay)  # Simulating processing time
           # else:
              #  print("No requests to process. Waiting for new requests...")
             #   time.sleep(1)
   # except KeyboardInterrupt:
       # print("Stopping request processing...")



In [ ]:
import csv

# Initialize a set to keep track of unique request identifiers
processed_requests = set()

def generate_request_id(request):
    """Generate a unique identifier for each request based on specific fields."""
    return f"{request['User ID']}-{request['Priority']}-{request['Flight Path']}-{request['Start Time']}-{request['Period']}"

def write_requests_to_csv(request, filename="trip_requests.csv"):
    fieldnames = ['User ID', 'Priority', 'Flight Path', 'Pitch Price', 'Start Time', 'Period']
    request_id = generate_request_id(request)

    # Check if the request has already been processed
    if request_id in processed_requests:
        print(f"Duplicate request for {request_id}, skipping...")
        return  # Skip this request

    # Add the request ID to the set of processed requests
    processed_requests.add(request_id)

    # Write the request to the CSV file
    try:
        with open(filename, 'x') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerow(request)
    except FileExistsError:
        with open(filename, 'a') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow(request)

# Updated process_requests function with CSV writing
def process_requests(request_queue, process_delay=2, filename="trip_requests.csv"):
    try:
        while True:
            if not request_queue.empty():
                request = request_queue.get()
                print(f"Processing request from User {request['User ID']}: {request}")
                write_requests_to_csv(request, filename)  # Write the request to a CSV file
                time.sleep(process_delay)  # Simulating processing time
            else:
                print("No requests to process. Waiting for new requests...")
                time.sleep(1)
    except KeyboardInterrupt:
        print("Stopping request processing...")


In [ ]:
# Create a queue to hold the trip requests
request_queue = queue.Queue()

# Start the trip request generator in a separate thread
generator_thread = threading.Thread(target=trip_request_generator, args=(request_queue,))
generator_thread.start()

# Start processing requests in the main thread
process_requests(request_queue)


Processing request from User 2: {'User ID': 2, 'Flight Path': 3, 'Priority': 1, 'Pitch Price': '£27.76', 'Start Time': '17:00', 'Period': 'on-peak'}
Processing request from User 3: {'User ID': 3, 'Flight Path': 4, 'Priority': 1, 'Pitch Price': '£31.94', 'Start Time': '17:00', 'Period': 'on-peak'}
Processing request from User 1: {'User ID': 1, 'Flight Path': 3, 'Priority': 2, 'Pitch Price': '£41.04', 'Start Time': '21:00', 'Period': 'on-peak'}
Processing request from User 2: {'User ID': 2, 'Flight Path': 4, 'Priority': 1, 'Pitch Price': '£22.83', 'Start Time': '21:00', 'Period': 'on-peak'}
Processing request from User 1: {'User ID': 1, 'Flight Path': 2, 'Priority': 1, 'Pitch Price': '£15.69', 'Start Time': '02:00', 'Period': 'off-peak'}
Processing request from User 3: {'User ID': 3, 'Flight Path': 2, 'Priority': 2, 'Pitch Price': '£19.37', 'Start Time': '12:00', 'Period': 'on-peak'}
Processing request from User 3: {'User ID': 3, 'Flight Path': 2, 'Priority': 2, 'Pitch Price': '£38.89', 